# Natural Language Processing Using RNNs and Attention

#### Generating Shakespeare using a character RNN

In [16]:
from tensorflow import keras
import numpy as np
import tensorflow as tf

In [17]:
shakespeare_url = 'https://homl.info/shakespeare'
filepath = keras.utils.get_file('shakespeare.txt', shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

In [18]:
#now encode every character as an integer

tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts([shakespeare_text])

In [19]:
tokenizer.texts_to_sequences(['First'])

[[20, 6, 9, 8, 3]]

In [20]:
tokenizer.sequences_to_texts([[20, 6, 9, 8, 3]])

['f i r s t']

In [21]:
max_id = len(tokenizer.word_index)

In [22]:
#encode the full text so each character is represented by its ID

[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1

In [23]:
#splitting the data set needs to be done carefully, since it is sequential

dataset_size = encoded.shape[0]
train_size = dataset_size * 90//100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

In [24]:
#we need to use the data sets window method to convert the long string of characters to smaller readable pieces.

n_steps = 100
window_length = n_steps + 1
dataset = dataset.window(window_length, shift=1, drop_remainder=True)

In [25]:
#flatten the data set for training

dataset = dataset.flat_map(lambda window: window.batch(window_length))

In [26]:
#now we can shuffle the windows 

batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

In [27]:
#we only havee 39 distinct characters so lets encode them as one hot vectors

dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))

dataset = dataset.prefetch(1)

In [28]:
for X_batch, Y_batch in dataset.take(1):
    print(X_batch.shape, Y_batch.shape)

(32, 100, 39) (32, 100)


In [29]:
#now the data has been prepared and we can move on to creating the model.

model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, input_shape=[None,max_id],
                    dropout=0.2),
    keras.layers.GRU(128, return_sequences=True,
                    dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation='softmax'))
])

In [30]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')
history = model.fit(dataset, epochs=20)

Epoch 1/20
31368/31368 [==============================] - 11527s 367ms/step - loss: 1.6202
Epoch 2/20
31368/31368 [==============================] - 2454s 78ms/step - loss: 1.5383
Epoch 3/20
31368/31368 [==============================] - 2406s 77ms/step - loss: 1.5175
Epoch 4/20
31368/31368 [==============================] - 2402s 77ms/step - loss: 1.5066
Epoch 5/20
31368/31368 [==============================] - 2413s 77ms/step - loss: 1.4988
Epoch 6/20
31368/31368 [==============================] - 2417s 77ms/step - loss: 1.4936
Epoch 7/20
31368/31368 [==============================] - 35558s 1s/step - loss: 1.4896
Epoch 8/20
31368/31368 [==============================] - 2494s 79ms/step - loss: 1.4863
Epoch 9/20
31368/31368 [==============================] - 2570s 82ms/step - loss: 1.4838
Epoch 10/20
31368/31368 [==============================] - 278355s 9s/step - loss: 1.4810
Epoch 11/20
31368/31368 [==============================] - 2622s 84ms/step - loss: 1.4789
Epoch 12/20
31368/

In [31]:
#This model predicts the next character written by Shakespeare. Preprocessing is required to feed text.

def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

In [33]:
#test it out with a simple sentence

X_new = preprocess(['How are yo'])
Y_pred = np.argmax(model.predict(X_new), axis=-1)
tokenizer.sequences_to_texts(Y_pred + 1)[0][-1]

1/1 [==============================] - 1s 882ms/step


'u'

In [40]:
def next_char(text, temperature=1):
    X_new = preprocess([text])
    Y_proba = model.predict(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(Y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

def complete_text(text, n_chars=100, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [51]:
#with these functions we can now print out some text.

print(complete_text('Who are you', temperature=1))

1/1 [==============================] - 0s 94ms/step
Who are you profumed 
we laysing more sisters to his reason.

katharina:
well i never kath rival him my wife, i


In [52]:
#This has been a demonstration of a stateless RNN. In a stateful RNN, the final state from processing one batch
#is used in the initial state for the next training batch.
#this only makes sense if each input sequnce in a batch starts where the last one left off.
#We need sequential, non-overlapping input sequences.

dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])
dataset = dataset.window(window_length, shift=n_steps, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(window_length))
dataset = dataset.batch(1)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
dataset = dataset.prefetch(1)

In [53]:
#Create Stateful RNN

model = keras.models.Sequential([
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                    dropout=0.2, recurrent_dropout=0.2,
                    batch_input_shape=[batch_size, None, max_id]),
    keras.layers.GRU(128, return_sequences=True, stateful=True,
                    dropout=0.2, recurrent_dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation='softmax'))
])

In [54]:
#create a small callback to reset the states

class ResetStatesCallback(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs):
        self.model.reset_states()

In [55]:
#commenting this out for now for time constraints

#model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')
#model.fit(dataset, epochs=50, callbacks=[ResetStatesCallback()])

In [58]:
#This has been a look at character level models. We can now look at a word level model.
#Using word level models is useful in Sentiment Analysis. IMDb data set is popular for sentiment analysis

(X_train, y_train), (X_test, y_test) = keras.datasets.imdb.load_data()

In [59]:
X_train[0][:10]

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65]

In [60]:
#The reviews have already been preprocessed. Can decode an individual review like this:

word_index = keras.datasets.imdb.get_word_index()
id_to_word = {id_ + 3: word for word, id_ in word_index.items()}
for id_, token in enumerate(('<pad>', '<sos>', '<unk>')):
    id_to_word[id_] = token

1641221/1641221 [==============================] - 0s 0us/step


In [61]:
' '.join([id_to_word[id_] for id_ in X_train[0][:10]])

'<sos> this film was just brilliant casting location scenery story'

In [64]:
#We can load the original reviews as byte strings from tensorflow

import tensorflow_datasets as tfds

datasets, info = tfds.load('imdb_reviews', as_supervised=True, with_info=True)
train_size = info.splits['train'].num_examples

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling ~\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incompleteZLCN4C\imdb_reviews-train.tfrecord*...…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling ~\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incompleteZLCN4C\imdb_reviews-test.tfrecord*...:…

Generating unsupervised examples...: 0 examples [00:00, ? examples/s]

Shuffling ~\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incompleteZLCN4C\imdb_reviews-unsupervised.tfrec…

Dataset imdb_reviews downloaded and prepared to ~\tensorflow_datasets\imdb_reviews\plain_text\1.0.0. Subsequent calls will reuse this data.


In [65]:
#write our preprocessing function to only keep the first 300 characters 
#uses regular expressions to replace any <br /> tages with spaces
#replaces any characters other than letters and quotes with spaces
#pads all reviews with a padding token to ensure the same length

def preprocess(X_batch, y_batch):
    X_batch = tf.strings.substr(X_batch, 0, 300)
    X_batch = tf.strings.regex_replace(X_batch, b"<br\\s*/?>", b" ")
    X_batch = tf.strings.regex_replace(X_batch, b"[^a-zA-Z']", b" ")
    X_batch = tf.strings.split(X_batch)
    return X_batch.to_tensor(default_value=b'<pad>'), y_batch

In [66]:
#Construct our vocab by running through the entire training set and counting the occurences of each word

from collections import Counter
vocabulary = Counter()
for X_batch, y_batch in datasets['train'].batch(32).map(preprocess):
    for review in X_batch:
        vocabulary.update(list(review.numpy()))

In [67]:
#check out the three most common words

vocabulary.most_common()[:3]

[(b'<pad>', 214309), (b'the', 61137), (b'a', 38564)]

In [68]:
#keep only the 10000 most common

vocab_size = 10000
truncated_vocabulary = [
    word for word, count in vocabulary.most_common()[:vocab_size]
]

In [69]:
words = tf.constant(truncated_vocabulary)
word_ids = tf.range(len(truncated_vocabulary), dtype=tf.int64)
vocab_init = tf.lookup.KeyValueTensorInitializer(words, word_ids)
num_oov_buckets = 1000
table = tf.lookup.StaticVocabularyTable(vocab_init, num_oov_buckets)

In [70]:
table.lookup(tf.constant([b'this movie was faaaaantastic'.split()]))

<tf.Tensor: shape=(1, 4), dtype=int64, numpy=array([[    9,    12,    11, 10791]], dtype=int64)>

In [71]:
#now we can create our training set using the preprocessing steps.

def encode_words(X_batch, y_batch):
    return table.lookup(X_batch), y_batch

train_set = datasets['train'].batch(32).map(preprocess)
train_set = train_set.map(encode_words).prefetch(1)

In [72]:
embed_size = 128
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size,
                          input_shape=[None]),
    keras.layers.GRU(128, return_sequences=True),
    keras.layers.GRU(128),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam',
             metrics=['accuracy'])

In [74]:
history = model.fit(train_set, epochs=50)

Epoch 1/50
782/782 [==============================] - 42s 54ms/step - loss: 0.0355 - accuracy: 0.9883
Epoch 2/50
782/782 [==============================] - 43s 55ms/step - loss: 0.0286 - accuracy: 0.9906
Epoch 3/50
782/782 [==============================] - 45s 57ms/step - loss: 0.0245 - accuracy: 0.9918
Epoch 4/50
782/782 [==============================] - 45s 57ms/step - loss: 0.0247 - accuracy: 0.9918
Epoch 5/50
782/782 [==============================] - 45s 57ms/step - loss: 0.0200 - accuracy: 0.9939
Epoch 6/50
782/782 [==============================] - 45s 57ms/step - loss: 0.0164 - accuracy: 0.9945
Epoch 7/50
782/782 [==============================] - 45s 57ms/step - loss: 0.0192 - accuracy: 0.9936
Epoch 8/50
782/782 [==============================] - 45s 58ms/step - loss: 0.0165 - accuracy: 0.9943
Epoch 9/50
782/782 [==============================] - 45s 58ms/step - loss: 0.0157 - accuracy: 0.9948
Epoch 10/50
782/782 [==============================] - 45s 58ms/step - loss: 0.014

In [75]:
#the model needs to know that the padding tokens should be ignored
#can be done simply by setting mask_zero=True

#The following model is the same as the previous one, except it is built with functional API and handles masking manually

K = keras.backend
inputs = keras.layers.Input(shape=[None])
mask = keras.layers.Lambda(lambda inputs: K.not_equal(inputs, 0))(inputs)
z = keras.layers.Embedding(vocab_size + num_oov_buckets, embed_size)(inputs)
z = keras.layers.GRU(128, return_sequences=True)(z, mask=mask)
z = keras.layers.GRU(128)(z, mask=mask)
outputs = keras.layers.Dense(1, activation='sigmoid')(z)
model = keras.Model(inputs=[inputs], outputs=[outputs])

In [79]:
#we can reuse pretrained embeddings from tensorflow

import tensorflow_hub as hub

model = keras.Sequential([
    hub.KerasLayer('https://tfhub.dev/google/tf2-preview/nnlm-en-dim50/1',
                  dtype=tf.string, input_shape=[], output_shape=[50]),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam',
             metrics=['accuracy'])

In [80]:
#by default, this is not trainable, but we can set trainable=True to fine tune for our task

#load the imdb data and train the model.

datasets, info = tfds.load('imdb_reviews', as_supervised=True, with_info=True)
train_size = info.splits['train'].num_examples
batch_size = 32
train_set = datasets['train'].batch(batch_size).prefetch(1)
history = model.fit(train_set, epochs=5)

Epoch 1/5
782/782 [==============================] - 3s 3ms/step - loss: 0.5527 - accuracy: 0.7212
Epoch 2/5
782/782 [==============================] - 2s 3ms/step - loss: 0.5164 - accuracy: 0.7468
Epoch 3/5
782/782 [==============================] - 2s 3ms/step - loss: 0.5111 - accuracy: 0.7497
Epoch 4/5
782/782 [==============================] - 2s 3ms/step - loss: 0.5074 - accuracy: 0.7510
Epoch 5/5
782/782 [==============================] - 2s 3ms/step - loss: 0.5040 - accuracy: 0.7536


In [83]:
#Next section will focus on neural machine translation

#This code is a basic encoder decoder model

import tensorflow_addons as tfa

encoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
decoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
sequence_lengths = keras.layers.Input(shape=[], dtype=np.int32)

embeddings = keras.layers.Embedding(vocab_size, embed_size)
encoder_embeddings = embeddings(encoder_inputs)
decoder_embeddings = embeddings(decoder_inputs)

encoder = keras.layers.LSTM(512, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_embeddings)
encoder_state = [state_h, state_c]

sampler = tfa.seq2seq.sampler.TrainingSampler()

decoder_cell = keras.layers.LSTMCell(512)
output_layer = keras.layers.Dense(vocab_size)
decoder = tfa.seq2seq.basic_decoder.BasicDecoder(decoder_cell, sampler,
                                                output_layer=output_layer)
final_outputs, final_state, final_sequence_lengths = decoder(
    decoder_embeddings, initial_state=encoder_state,
    sequence_length=sequence_lengths)
Y_proba = tf.nn.softmax(final_outputs.rnn_output)

model = keras.Model(inputs=[encoder_inputs, decoder_inputs, sequence_lengths],
                   outputs=[Y_proba])

In [84]:
#durign NLP, it is often useful to implement a bidirectional recurrent layer

#keras.layers.Bidirectional(keras.layers.GRU(10, return_sequences=True))